# Extracting onset and clinical variables from clinical databases

In [2]:
import pandas as pd
import seaborn as sns
import os
import numpy as np
from gsd_pipeline.dataset_tools.filter_for_clinical_param import filter_for_clinical_param
from gsd_pipeline.clinical_data.from_2018.extract_clinical_outcomes import extract_clinical_outcomes as extract_2018
from gsd_pipeline.clinical_data.years_2015_2016_2017.extract_clinical_outcomes import extract_clinical_outcomes as extract_2015
from gsd_pipeline.clinical_data.join_multi_annual_outcome_df import join_multi_annual_outcome_df

In [26]:
variable_names = '/Users/jk1/OneDrive - unige.ch/stroke_research/scope/variables/onset/onset_scope_variable_selection.xlsx'
patient_ids_2018 = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/imaging_keys/2018/patient_ids_keys_2018.xlsx'
clinical_data_2018 = '/Users/jk1/temp/scope_clinical_data_extraction/SSR_cases_of_2018_(Adm,_Hosp_and_FU).xlsx'

patient_ids_2015_2016_2017 = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/imaging_keys/2015-2017/anonymisation_key_pCT_2016_2017.xlsx'
clinical_data_2015_2016_2017 = '/Users/jk1/temp/scope_clinical_data_extraction/190419_Données 2015-16-17.xlsx'

variable_names = pd.read_excel(variable_names, header=None)

### Extracting data for 2018

In [11]:
variable_names_2018 = variable_names.loc[0,1:].values.tolist()

extract_2018(
    patient_ids_2018,
    clinical_data_2018, id_sheet='Sheet1',
    info_sheet='Export cases registered in.',
    anonymise=False,
    selected_outcomes = variable_names_2018
)


Output may contain duplicates, please remove them manually as not all duplicate entries are the same.


### Extracting data for 2015-2017

In [15]:
variable_names_2015_2016_2017 = variable_names.loc[1,1:].dropna().values.tolist()

extract_2015(
        patient_ids_2015_2016_2017,
        clinical_data_2015_2016_2017,
        id_sheet = 'Sheet1', info_sheet = 'Sheet1 (2)', anonymise=False,
        selected_outcomes = variable_names_2015_2016_2017)


For 2015-2017 patients, additional data is added manually to mRS follow-up

For 2018 patients some 1st imaging dates are completed manually by checking DICOMs

## Onset timing variable creation
Adding onset to imaging variable for 2018 (added manually to 2015-2017 because of inconsistent data)

In [22]:
extracted_2018_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/clinical_data/2018/scope_extracted_completed_variables/completed_extracted_clinical_outcomes_SSR_cases_of_2018_(Adm,_Hosp_and_FU).xlsx'
extracted_2018_df = pd.read_excel(extracted_2018_path)

In [23]:
onset_date = pd.to_datetime(pd.to_datetime(extracted_2018_df['Onset date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + extracted_2018_df['Onset time'], format='%d-%m-%Y %H:%M')

imaging_date = pd.to_datetime(pd.to_datetime(extracted_2018_df['1st brain imaging date'], format='%Y%m%d').dt.strftime('%d-%m-%Y') \
                                        + ' ' + extracted_2018_df['1st brain imaging time'], format='%d-%m-%Y %H:%M')

extracted_2018_df['onset_to_imaging'] = (imaging_date-onset_date).dt.total_seconds().div(60)

In [24]:
extracted_2018_df['onset_date_reformatted'] = onset_date
extracted_2018_df['imaging_date_reformatted'] = imaging_date

In [25]:
extracted_2018_df.to_excel(extracted_2018_path)

### Joining data

In [27]:
modified_variable_names_2015_2016_2017 = variable_names.loc[2,1:].dropna().values.tolist()
modified_variable_names_2018 = variable_names.loc[3,1:].dropna().values.tolist()

In [29]:
complemented_extracted_2015_2016_2017_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/clinical_data/2015-2017/scope_extracted_variables_data/onset_selected_variables/complemented_extracted_clinical_outcomes_190419_Données 2015-16-17.xlsx'

In [30]:
join_multi_annual_outcome_df(extracted_2018_path,
                             complemented_extracted_2015_2016_2017_path,
                             'pid',
                             'anonymised_id',
                             modified_variable_names_2018,
                             modified_variable_names_2015_2016_2017)

## Data curation

In [44]:
joined_data_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/clinical_data/multi_annual_joined_data/scope_joined_variables/onset_prediction/joined_anon_outcome_df.xlsx'
joined_data = pd.read_excel(joined_data_path)

In [45]:
joined_data['onset_to_imaging'].isnull().sum()

3

Exclude patients with missing onset to imaging

In [46]:
joined_data = joined_data[joined_data['onset_to_imaging'].notnull()]


In [47]:
joined_data.isnull().sum(axis = 0)

Unnamed: 0                        0
pid                               0
Age (calc.)                       0
Sex                               0
Time of symptom onset known       1
Referral                          0
Prestroke disability (Rankin)     0
NIH on admission                  2
1st syst. bp                      0
1st diast. bp                     0
BMI                              17
1st glucose                       4
1st creatinine                    2
Antiplatelet drugs                0
Anticoagulants                    1
IVT with rtPA                     1
IAT                               2
MedHist Stroke                    1
MedHist TIA                       1
MedHist ICH                       1
MedHist Hypertension              0
MedHist Diabetes                  0
MedHist Hyperlipidemia            0
MedHist Smoking                   0
MedHist Atrial Fibr.              0
3M mRS                           35
Door to image (min.)             12
onset_to_imaging            

Curate BMI variable

In [48]:
joined_data.loc[joined_data['BMI'] == '?', 'BMI'] = np.nan
joined_data['BMI'] = joined_data['BMI'].astype('float')

Curate Sex variable

In [49]:
joined_data.loc[joined_data['Sex'] == 'F', 'Sex'] = 'Female'
joined_data.loc[joined_data['Sex'] == 'M', 'Sex'] = 'Male'
joined_data['Sex'].value_counts()

Male      112
Female     90
Name: Sex, dtype: int64

Curate Referral variable

In [50]:
joined_data.loc[joined_data['Referral'] == 'Emergency service (144)', 'Referral'] = 'Emergency service'
joined_data.loc[joined_data['Referral'] == 'samu', 'Referral'] = 'Emergency service'
joined_data.loc[joined_data['Referral'] == 'General practionner', 'Referral'] = 'general practitioner'
joined_data.loc[joined_data['Referral'] == 'in hospital stroke', 'Referral'] = 'in-hospital event'
joined_data['Referral'] = joined_data['Referral'].str.lower()
joined_data['Referral'].value_counts()

emergency service       152
self referral            20
other hospital           18
in-hospital event         9
general practitioner      2
samu                      1
Name: Referral, dtype: int64

Curate IVT treatment variable

In [51]:
joined_data['IVT with rtPA'] = joined_data['IVT with rtPA'].str.strip()
joined_data.loc[joined_data['IVT with rtPA'] == 'oui', 'IVT with rtPA'] = 'yes'
joined_data.loc[joined_data['IVT with rtPA'] == 'started before admission', 'IVT with rtPA'] = 'yes'
joined_data.loc[joined_data['IVT with rtPA'] == 'non', 'IVT with rtPA'] = 'no'
joined_data['IVT with rtPA'].value_counts()

yes    165
no      36
Name: IVT with rtPA, dtype: int64

Strip whitespaces in all medical history columns

In [52]:
filter_col = [col for col in joined_data if col.startswith('MedHist')]
joined_data[filter_col] = joined_data[filter_col].apply(lambda column: column.str.strip())
joined_data['MedHist Hyperlipidemia'].value_counts()

no     123
yes     79
Name: MedHist Hyperlipidemia, dtype: int64

Convert categorical variables to integers

*Note: missing variables are encoded as -1 -> there are then removed again*

In [53]:
char_cols = joined_data.dtypes.pipe(lambda x: x[x == 'object']).index
# Ignore pid column
char_cols = char_cols.drop('pid')
label_mapping = {}

for c in char_cols:
    joined_data[c], label_mapping[c] = pd.factorize(joined_data[c])
    joined_data.loc[joined_data[c] < 0, c] = np.nan

#### Action required
Verify for duplicates (these should be removed before converting categories to binaries)

In [54]:
label_mapping


{'Sex': Index(['Female', 'Male'], dtype='object'),
 'Time of symptom onset known': Index(['yes', 'wake up', 'no'], dtype='object'),
 'Referral': Index(['emergency service', 'in-hospital event', 'self referral',
        'other hospital', 'samu', 'general practitioner'],
       dtype='object'),
 'Antiplatelet drugs': Index(['no', 'yes'], dtype='object'),
 'Anticoagulants': Index(['no', 'yes'], dtype='object'),
 'IVT with rtPA': Index(['yes', 'no'], dtype='object'),
 'IAT': Index(['no', 'yes'], dtype='object'),
 'MedHist Stroke': Index(['no', 'yes'], dtype='object'),
 'MedHist TIA': Index(['no', 'yes'], dtype='object'),
 'MedHist ICH': Index(['no', 'yes'], dtype='object'),
 'MedHist Hypertension': Index(['no', 'yes'], dtype='object'),
 'MedHist Diabetes': Index(['no', 'yes'], dtype='object'),
 'MedHist Hyperlipidemia': Index(['yes', 'no'], dtype='object'),
 'MedHist Smoking': Index(['no', 'yes'], dtype='object'),
 'MedHist Atrial Fibr.': Index(['no', 'yes'], dtype='object')}

In [5]:
# save curated data
curated_data_path = os.path.join(os.path.dirname(joined_data_path), 'curated_completed_joined_anon_outcome_df.xlsx')
joined_data.to_excel(curated_data_path)

NameError: name 'joined_data_path' is not defined

## Restrict to patients with imaging data available

In [3]:
# imaging_dataset_path = '/Users/jk1/stroke_datasets/dataset_files/perfusion_data_sets/noGT_datasets/noGT_pmaps_15-19_dataset.npz'
imaging_dataset_path = '/Users/jk1/stroke_datasets/dataset_files/perfusion_data_sets/with_ncct_dataset.npz'
ids = np.load(imaging_dataset_path, allow_pickle=True)['ids']

In [7]:
data_with_imaging = joined_data[joined_data['pid'].isin(ids)]
data_with_imaging.to_excel(os.path.join(os.path.dirname(joined_data_path), 'with_imaging_curated_completed_joined_anon_outcome_df.xlsx'))

NameError: name 'joined_data' is not defined

## Filter imaging dataset for subjects having all clinical variables

In [4]:
data_with_imaging_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/clinical_data/multi_annual_joined_data/scope_joined_variables/onset_prediction/with_imaging_curated_completed_joined_anon_outcome_df.xlsx'
data_with_imaging = pd.read_excel(data_with_imaging_path)

In [6]:
filter_for_clinical_param(imaging_dataset_path, data_with_imaging_path, 'onset_to_imaging','pid')

Loading a total of 144 subjects.
Sequences used: {'ct_sequences': ['wcoreg_Tmax', 'wcoreg_CBF', 'wcoreg_MTT', 'wcoreg_CBV', 'wreor_SPC_301mm_Std'], 'ct_label_sequences': ['masked_wcoreg_VOI'], 'mri_sequences': [], 'mri_label_sequences': []}
0 subjects had been excluded.
subj-5ca2adc0 not found in clinical database. Will be removed.
subj-848a19b9 not found in clinical database. Will be removed.
subj-a84863e6 not found in clinical database. Will be removed.
subj-dcdfb2af not found in clinical database. Will be removed.
subj-e65bbcbe not found in clinical database. Will be removed.
subj-3e0e8660 not found in clinical database. Will be removed.
Saving a total of 138 subjects.


## Binarize onset variable

In [11]:
data_with_imaging_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_dataset/clinical_data/multi_annual_joined_data/scope_joined_variables/onset_prediction/with_imaging_curated_completed_joined_anon_outcome_df.xlsx'
data_with_imaging = pd.read_excel(data_with_imaging_path)

In [12]:
data_with_imaging['onset_to_imaging_0-4.5'] = (data_with_imaging['onset_to_imaging'] > (4.5*60)).astype(int)

In [13]:
data_with_imaging.to_excel(data_with_imaging_path)

In [13]:
data_with_imaging['IAT'].value_counts()

0.0    104
1.0     87
Name: IAT, dtype: int64

In [14]:
data_with_imaging['IVT with rtPA'].value_counts()

0.0    159
1.0     33
Name: IVT with rtPA, dtype: int64

In [15]:
len(data_with_imaging[(data_with_imaging['IVT with rtPA'] == 0) & (data_with_imaging['IAT'] == 0)])

104